# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
db_series = client.series

In [3]:
print(client.list_database_names())

['admin', 'config', 'local', 'series']


In [4]:
collection_youtube = db_series["youtube"]
collection_youtube.delete_many({})

In [5]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

#### Youtube videos

In [6]:
def str_to_datetime(string):
    if len(string) == 5:
        return string
    
    splited = string.split('.')
    corrected = [ '0' + e if len(e) == 1 else e for e in splited]
    return corrected[0] + '.' + corrected[1]
    
df_youtube["date"] = pd.to_datetime(df_youtube["date"].astype('string').apply(str_to_datetime), format="%d.%m")

In [7]:
df_youtube["tags"] = df_youtube["tags"].str.split('|')

In [8]:
df_youtube = df_youtube.drop_duplicates(subset="video_id")
df_youtube = df_youtube.rename(columns={"video_id":"_id"})

In [9]:
df_youtube.head()

,_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,1900-09-13
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,1900-09-13
2,cLdxuaxaQwc,My Response,PewDiePie,22,[[none]],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,1900-09-13
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,"[apple iphone x hands on, Apple iPhone X, iPho...",2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,1900-09-13
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,"[jacksfilms, parody, parodies, iphone, iphone ...",1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,1900-09-13


#### Youtube video categories

In [10]:
import json

with open('./data/US_category_id.json') as json_file:
    data = json.load(json_file)

In [11]:
categories = data["items"]

In [12]:
categories[0]

{'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
 'id': '1',
 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation',
  'assignable': True}}

In [13]:
df_youtube["category"] = "Unknown"

In [14]:
def find_category(category_id, categories=categories):
    for category in categories:
        if int(category["id"]) == int(category_id):
            return category["snippet"]["title"]
    return "Unknown"


In [15]:
df_youtube["category"] = df_youtube["category_id"].apply(lambda x: find_category(x))

In [16]:
df_youtube.head()

,_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,1900-09-13,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,1900-09-13,Science & Technology
2,cLdxuaxaQwc,My Response,PewDiePie,22,[[none]],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,1900-09-13,People & Blogs
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,"[apple iphone x hands on, Apple iPhone X, iPho...",2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,1900-09-13,Science & Technology
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,"[jacksfilms, parody, parodies, iphone, iphone ...",1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,1900-09-13,Comedy


### Importer les données

In [17]:
collection_youtube.insert_many(df_youtube.to_dict('records'))

In [18]:
db_series.list_collection_names()

['youtube', 'kickstarter', '{query :{gender:"Male"}, out:"sumAge"}', 'peaky']

In [19]:
collection_youtube.find_one()

{'_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': datetime.datetime(1900, 9, 13, 0, 0),
 'category': 'Entertainment'}

___

## Question 1  

### Récupérer toutes les vidéos d'Apple

In [20]:
cursor = collection_youtube.find({"channel_title":"Apple"})

In [21]:
for e in cursor:
    print(e["title"])
    print("----------")

iPhone X — Introducing iPhone X — Apple
----------
Apple Watch Series 3 + Apple Music — Roll — Apple
----------
iPhone 8 Plus — Portraits of Her  — Apple
----------


## Question 2

### Compter le nombre de catégories différentes

In [22]:
len(collection_youtube.distinct("category"))

16

## Question 3

### Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update. 

In [23]:
collection_youtube.find_one()["tags"]

['logan paul vlog',
 'logan paul',
 'logan',
 'paul',
 'olympics',
 'logan paul youtube',
 'vlog',
 'daily',
 'comedy',
 'hollywood',
 'parrot',
 'maverick',
 'bird',
 'maverick clothes',
 'diamond play button',
 'logan paul diamond play button',
 '10M subscribers',
 'logan paul 1 year vlogging',
 '1 year vlog',
 'dwarf mamba play button',
 'logan paul history',
 'youtube history',
 '10M',
 '10M plaque',
 'youtube button',
 'diamond button',
 'logang',
 'logang 4 life']

## Question 4

### Récupérer les vidéos les plus vues.

In [24]:
n = 3
cursor = collection_youtube.find({}).limit(n).sort([("views", -1)])

for e in cursor:
    print("TITLE    : ", e["title"])
    print("VIEWS    :", e["views"])
    print("CATEGORY :", e["category"])
    print()

TITLE    :  ZAYN - Dusk Till Dawn ft. Sia
VIEWS    : 36323498
CATEGORY : Music

TITLE    :  BTS (방탄소년단) 'DNA' Official MV
VIEWS    : 20565649
CATEGORY : Music

TITLE    :  Primitive Technology: Mud Bricks
VIEWS    : 14068244
CATEGORY : People & Blogs



## Question 5

### Compter le nombre moyen de vues en fonction de la catégorie.

In [25]:
cursor = collection_youtube.aggregate([{"$group" : {"_id" : "$category", "AvgViews" : {"$avg" : "$views"}}}])


for e in cursor:
    print(e["_id"])
    print(" ->", round(e["AvgViews"]))
    print()

Pets & Animals
 -> 412382

Education
 -> 368229

Travel & Events
 -> 300664

Autos & Vehicles
 -> 415129

Sports
 -> 470686

Gaming
 -> 492390

People & Blogs
 -> 597891

Film & Animation
 -> 558289

Music
 -> 697599

Science & Technology
 -> 515854

Comedy
 -> 828485

Howto & Style
 -> 405748

News & Politics
 -> 374883

Entertainment
 -> 555071

Nonprofits & Activism
 -> 572023

Shows
 -> 8259



## Question 6 

### Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [26]:
cursor = collection_youtube.aggregate(
    [
        {"$group" : {"_id" : "$channel_title", "AvgLike" : {"$avg" : "$likes"}}},
        {"$sort" : {"AvgLike" : -1}}
    ]
)

next(cursor)

{'_id': 'ZaynVEVO', 'AvgLike': 1431683.0}